In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('../Data/bonbanh.csv')

# Gearbox

In [3]:
data['gear'].unique()

array(['Số tự động', 'Số tay'], dtype=object)

In [4]:
id1 = data[data['gear'] =='Số tự động'].index
id2 = data[data['gear'] =='Số tay'].index

In [5]:
data.loc[id1,'gear'] = 'Tự động'
data.loc[id2,'gear'] = 'Số sàn'

In [6]:
data['gear'].unique()

array(['Tự động', 'Số sàn'], dtype=object)

# Engine

In [7]:
idXeDien = data[data['engine'].str.contains('Điện')].index

In [8]:
data.drop(index=idXeDien,inplace=True)
data.reset_index(drop=True, inplace=True)

In [9]:
id1 = data[data.engine.str[0:1] =='X'].index
id2 = data[data.engine.str[0:1] =='D'].index
id3 = data[data.engine.str[0:1] =='H'].index

In [10]:
data.loc[id1,'fuel_x'] = 'Xăng'
data.loc[id2,'fuel_x'] = 'Dầu'
data.loc[id3,'fuel_x'] = 'Động cơ Hybrid'

In [11]:
data['fuel_x'].unique()

array(['Xăng', 'Dầu', 'Động cơ Hybrid'], dtype=object)

# Car Origin

In [12]:
data.car_origin.unique()

array(['Lắp ráp trong nước', 'Nhập khẩu', nan], dtype=object)

In [13]:
id1 = data[data['car_origin'] =='Lắp ráp trong nước'].index
id2 = data[data['car_origin'] =='Nhập khẩu'].index

In [14]:
data['origin'] = ''
data.loc[id1,'origin'] = 'Việt Nam'
data.loc[id2,'origin'] = 'Nước khác'

In [15]:
data['origin'].unique()

array(['Việt Nam', 'Nước khác', ''], dtype=object)

# Car Type

In [16]:
data.car_type.unique()

array(['Crossover', 'Bán tải / Pickup', 'SUV', 'Truck', 'Sedan',
       'Hatchback', 'Van/Minivan', 'Convertible/Cabriolet', 'Coupe', nan,
       'Wagon'], dtype=object)

In [17]:
tmp_dict = {
    'Hatchback':'Hatchback',
    'Crossover':'SUV / Cross over',
    'SUV':'SUV / Cross over',
    'Bán tải / Pickup':'Pick-up (bán tải)',
    'Truck':'Kiểu dáng khác',
    'Sedan':'Sedan',
    'Van/Minivan':'Van',
    'Convertible/Cabriolet':'Kiểu dáng khác',
    'Coupe':'Coupe (2 cửa)',
    'Wagon':'Sedan'
}

In [18]:
data['style']=''
for x in tmp_dict:
    id = data[data['car_type'] == x].index
    data.loc[id,'style'] = tmp_dict[x]

# Brand name

In [19]:
# data.rename(columns={'model':'car_model'},inplace=True)

In [20]:
from tqdm import tqdm

In [21]:
data['brand_name'] = ''
for x in tqdm(range(data.shape[0])):
    setA = set(str(data.loc[x,'cname']).split())
    setB = set(str(data.loc[x,'model']).split())
    res = ' '.join(setA - setB)
    data.loc[x,'brand_name'] = res

100%|██████████| 29691/29691 [00:19<00:00, 1524.97it/s]


In [22]:
id1 = data[data['brand_name'] =='Benz Mercedes'].index
data.loc[id1,'brand_name'] = 'Mercedes Benz'

# Car Model

In [23]:
tmp_brand_name = pd.read_csv('../util/Data/brand_name.csv')
tmp_car_model = pd.read_csv('../util/Data/car_model.csv')

In [24]:
tmp_dict = {
    'Cx3':'CX 3',
    'CX5':'CX 5',
    'CX8':'CX 8',
    'CX9':'CX 9',
    'I10':'Grand i10',
    'Lux A 2.0':'Lux',
    'Lux SA 2.0':'Lux',
    'MU-X':'Mu X',
    'VF E34':'VFe34'
}

In [25]:
for x in tmp_dict:
    id = data[data['model'] == x].index
    data.loc[id,'model'] = tmp_dict[x]

# Region Name

In [26]:
tmp_dict = {
    'Bà Rịa Vũng Tàu':'Bà Rịa - Vũng Tàu',
    'TP HCM':'Tp Hồ Chí Minh',
    'CX8':'CX 8',
    'CX9':'CX 9',
    'Đăk Lăk':'Đắk Lắk',
    'Đăk Nông':'Đắk Nông'
}

In [27]:
data['region_name'] = data['seller_location']
for x in tmp_dict:
    id = data[data['seller_location'] == x].index
    data.loc[id,'region_name'] = tmp_dict[x]

In [32]:
cols= ['url','description','origin','status','car_mileage','car_color','car_doors','car_seats','gear',
      'car_year','car_price','style','createdAt','model','fuel_x',
       'brand_name','region_name']
data[cols].iloc[0]

# Status

In [39]:
id1 = data[data.status =='Xe đã dùng'].index
id2 = data[data.status =='Xe mới'].index

In [40]:
data.loc[id1,'status'] = 'Đã sử dụng'
data.loc[id2,'status'] = 'Mới'

In [55]:
cols= ['description','origin','status','car_mileage','car_color','car_seats','gear',
      'car_year','car_price','style','createdAt','model','fuel_x',
       'brand_name','region_name']
last_res = data[cols]

In [56]:
last_res.columns

Index(['description', 'origin', 'status', 'car_mileage', 'car_color',
       'car_seats', 'gear', 'car_year', 'car_price', 'style', 'createdAt',
       'model', 'fuel_x', 'brand_name', 'region_name'],
      dtype='object')

In [61]:
last_res.to_csv('../Data/bonbanh_integrate.csv',index=False)